In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df_cc = pd.read_excel('cc61_df3_vn.xlsx')
df_xray = pd.read_excel('Xray (2017-2018).xlsx')

In [3]:
list(df_cc.columns)

['vn',
 'vstdate',
 'vsttime',
 'sex',
 'age_y',
 'cc',
 'bps',
 'bpd',
 'pulse',
 'bw',
 'height',
 'temperature',
 'department',
 'PDX (1)',
 'Comorbidity (2)',
 'Complication (3)',
 'Other (4)',
 'External Cause (5)',
 'clean_symtom(noun)',
 'clean_symptom2(synonym)']

In [4]:
# dropping null value columns to avoid errors
df_cc.dropna(subset=['clean_symptom2(synonym)', 'bps', 'bpd', 'pulse', 'bw', 'height', 'temperature'],inplace = True)
df_cc.head()

,vn,vstdate,vsttime,sex,age_y,cc,bps,bpd,pulse,bw,height,temperature,department,PDX (1),Comorbidity (2),Complication (3),Other (4),External Cause (5),clean_symtom(noun),clean_symptom2(synonym)
0,610611111904,2018-06-11,11:19:04,หญิง,23.0,P - LAST - LMP 23/03/02561* 4 วัน ตั้งครร...,86.0,59.0,85.0,49.4,160.0,36.5,จุดซักประวัติห้องตรวจโรคนรีเวช,O020,NaN,NaN,NaN,NaN,ใช้สารเสพติด ตั้งครรภ์,ตั้งครรภ์/ยาเสพติด
1,610508143131,2018-05-08,14:31:31,หญิง,22.0,G1P 0 LAST- ปี LMP - คุมกำเนิด - วันนี...,123.0,72.0,130.0,61.1,165.0,36.5,จุดซักประวัติห้องตรวจโรคนรีเวช,A630,NaN,NaN,Z133,NaN,ฝากครรภ์ คุมกำเนิด ตั้งครรภ์,ฝากครรภ์/ตั้งครรภ์/กำเนิด
2,601101191449,2017-11-01,19:14:49,หญิง,51.0,รับจาก ER เวลา 19.32 น มีไข้ ไอมีเสมหะ ป...,175.0,99.0,114.0,0.0,156.0,38.5,คลินิกนอกเวลา1,J110,NaN,NaN,NaN,NaN,ปวดศีรษะ ชา ไข้ ไอ เสมหะ,ไข้/ไอ/ปวดศีรษะ/ชา/ปวด/เสมหะ
3,610926001048,2018-09-26,00:10:48,หญิง,52.0,ผู้ป่วยแจ้งว่า มีอาการปวดท้อง แน่นใต้ลิ้นปี่ เ...,170.0,110.0,64.0,86.0,154.0,36.6,ห้องEmergency Room (ER),K30,NaN,NaN,NaN,NaN,แน่นใต้ลิ้นปี่ ปวดท้อง,ปวด/แน่น/ปวดท้อง/แน่นใต้ลิ้นปี่
6,610326142345,2018-03-26,14:23:45,หญิง,51.0,ปวดขาพับข้างซ้ายร้าวลงขาซ้ายเป็นมา 1 อาทิตย์,153.0,92.0,67.0,86.0,154.0,36.5,จุดซักประวัติห้องตรวจโรคกระดูก,M6596,NaN,NaN,NaN,NaN,ร้าว ปวดขา,ปวด/ปวดร้าว/ปวดขา


In [5]:
list(df_cc.columns)

['vn',
 'vstdate',
 'vsttime',
 'sex',
 'age_y',
 'cc',
 'bps',
 'bpd',
 'pulse',
 'bw',
 'height',
 'temperature',
 'department',
 'PDX (1)',
 'Comorbidity (2)',
 'Complication (3)',
 'Other (4)',
 'External Cause (5)',
 'clean_symtom(noun)',
 'clean_symptom2(synonym)']

In [49]:
df_cc = df_cc.drop(df_cc[df_cc.bw < 10].index, inplace=True)

In [50]:
df_cc = df_cc.drop(df_cc[df_cc.height < 20].index, inplace=True)

AttributeError: 'NoneType' object has no attribute 'drop'

In [9]:
df_cc = df_cc.drop(df_cc[df_cc.temperature < 20].index, inplace=True)

In [10]:
df_xray['xray_items'] = df_xray['xray_list'].str.split(",", n= -1)
df_xray.head()

,vn,vstdate,vsttime,xray_items_code,xray_list,xray_items
0,601001011504,2017-10-01,01:15:04,12.0,Chest Ap,[Chest Ap]
1,601001011504,2017-10-01,01:15:04,66.0,Chest Ap,[Chest Ap]
2,601001011504,2017-10-01,01:15:04,66.0,Chest Ap,[Chest Ap]
3,601001012739,2017-10-01,01:27:39,1.0,"Skull AP,lat,Ankle Lt AP,lat,Ankle Mortise Lt","[Skull AP, lat, Ankle Lt AP, lat, Ankle Mortis..."
4,601001012739,2017-10-01,01:27:39,59.0,"Skull AP,lat,Ankle Lt AP,lat,Ankle Mortise Lt","[Skull AP, lat, Ankle Lt AP, lat, Ankle Mortis..."


In [11]:
df_cc['symptoms'] = df_cc['clean_symptom2(synonym)'].str.split("/", n= -1)
df_cc.head()

,vn,vstdate,vsttime,sex,age_y,cc,bps,bpd,pulse,bw,...,temperature,department,PDX (1),Comorbidity (2),Complication (3),Other (4),External Cause (5),clean_symtom(noun),clean_symptom2(synonym),symptoms
0,610611111904,2018-06-11,11:19:04,หญิง,23.0,P - LAST - LMP 23/03/02561* 4 วัน ตั้งครร...,86.0,59.0,85.0,49.4,...,36.5,จุดซักประวัติห้องตรวจโรคนรีเวช,O020,NaN,NaN,NaN,NaN,ใช้สารเสพติด ตั้งครรภ์,ตั้งครรภ์/ยาเสพติด,"[ตั้งครรภ์, ยาเสพติด]"
1,610508143131,2018-05-08,14:31:31,หญิง,22.0,G1P 0 LAST- ปี LMP - คุมกำเนิด - วันนี...,123.0,72.0,130.0,61.1,...,36.5,จุดซักประวัติห้องตรวจโรคนรีเวช,A630,NaN,NaN,Z133,NaN,ฝากครรภ์ คุมกำเนิด ตั้งครรภ์,ฝากครรภ์/ตั้งครรภ์/กำเนิด,"[ฝากครรภ์, ตั้งครรภ์, กำเนิด]"
3,610926001048,2018-09-26,00:10:48,หญิง,52.0,ผู้ป่วยแจ้งว่า มีอาการปวดท้อง แน่นใต้ลิ้นปี่ เ...,170.0,110.0,64.0,86.0,...,36.6,ห้องEmergency Room (ER),K30,NaN,NaN,NaN,NaN,แน่นใต้ลิ้นปี่ ปวดท้อง,ปวด/แน่น/ปวดท้อง/แน่นใต้ลิ้นปี่,"[ปวด, แน่น, ปวดท้อง, แน่นใต้ลิ้นปี่]"
6,610326142345,2018-03-26,14:23:45,หญิง,51.0,ปวดขาพับข้างซ้ายร้าวลงขาซ้ายเป็นมา 1 อาทิตย์,153.0,92.0,67.0,86.0,...,36.5,จุดซักประวัติห้องตรวจโรคกระดูก,M6596,NaN,NaN,NaN,NaN,ร้าว ปวดขา,ปวด/ปวดร้าว/ปวดขา,"[ปวด, ปวดร้าว, ปวดขา]"
7,610427095914,2018-04-27,09:59:14,หญิง,51.0,ไข้ ปวดเมื่อยตามตัว มีน้ำมูก อ่อนเพลีย ลูกเ...,150.0,85.0,79.0,86.0,...,36.9,จุดซักประวัติตรวจโรคทั่วไป,J111,NaN,NaN,NaN,NaN,น้ำมูก ไข้ ปวดเมื่อย อ่อนเพลีย,ไข้/ปวดเมื่อย/ปวด/น้ำมูก/อ่อนเพลีย,"[ไข้, ปวดเมื่อย, ปวด, น้ำมูก, อ่อนเพลีย]"


In [12]:
df_cc.rename(columns={'PDX (1)': 'PDX'}, inplace=True)

In [14]:
df_cc_new = df_cc[['vn', 'sex', 'age_y', 'bps', 'bpd', 'pulse', 'bw', 'height', 'temperature', 'PDX', 'symptoms']]
df_cc_new.head()

,vn,sex,age_y,bps,bpd,pulse,bw,height,temperature,PDX,symptoms
0,610611111904,หญิง,23.0,86.0,59.0,85.0,49.4,160.0,36.5,O020,"[ตั้งครรภ์, ยาเสพติด]"
1,610508143131,หญิง,22.0,123.0,72.0,130.0,61.1,165.0,36.5,A630,"[ฝากครรภ์, ตั้งครรภ์, กำเนิด]"
3,610926001048,หญิง,52.0,170.0,110.0,64.0,86.0,154.0,36.6,K30,"[ปวด, แน่น, ปวดท้อง, แน่นใต้ลิ้นปี่]"
6,610326142345,หญิง,51.0,153.0,92.0,67.0,86.0,154.0,36.5,M6596,"[ปวด, ปวดร้าว, ปวดขา]"
7,610427095914,หญิง,51.0,150.0,85.0,79.0,86.0,154.0,36.9,J111,"[ไข้, ปวดเมื่อย, ปวด, น้ำมูก, อ่อนเพลีย]"


In [15]:
df_xray_new = df_xray[['vn', 'xray_items']]
df_xray_new.head()

,vn,xray_items
0,601001011504,[Chest Ap]
1,601001011504,[Chest Ap]
2,601001011504,[Chest Ap]
3,601001012739,"[Skull AP, lat, Ankle Lt AP, lat, Ankle Mortis..."
4,601001012739,"[Skull AP, lat, Ankle Lt AP, lat, Ankle Mortis..."


In [16]:
df = pd.merge(df_cc_new, df_xray_new, how='inner', on='vn')
df.head()

,vn,sex,age_y,bps,bpd,pulse,bw,height,temperature,PDX,symptoms,xray_items
0,610926001048,หญิง,52.0,170.0,110.0,64.0,86.0,154.0,36.6,K30,"[ปวด, แน่น, ปวดท้อง, แน่นใต้ลิ้นปี่]",[Acute Abdomen]
1,601013203924,หญิง,51.0,150.0,94.0,74.0,86.0,156.0,36.4,K30,[เหนื่อย],[Chest PA Upright]
2,610330191455,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,"[ปวด, ปวดเอว]","[LS-spine AP, lat, Acute Abdomen]"
3,610330191455,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,"[ปวด, ปวดเอว]","[LS-spine AP, lat, Acute Abdomen]"
4,610627072254,หญิง,61.0,124.0,74.0,64.0,76.0,155.0,36.0,M70,"[ปวด, ปวดเข่า, เจ็บ, -]","[Knee Lt AP, lat standing, Knee Rt AP, lat sta..."


In [18]:
df.shape

(20257, 12)

In [20]:
df['symptoms'] = df.symptoms.apply("/".join)
df['xray_items'] = df.xray_items.apply("/".join)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
# TFIDF - symptoms
def slash_token(msg):
    return msg.split('/')

In [23]:
type(df['symptoms'][1])

str

In [24]:
vectorizer = TfidfVectorizer(tokenizer=slash_token)
vectorizer.fit(df['symptoms'])
vector = vectorizer.transform(df['symptoms'])

/Users/phrare/patient_clustering/patient/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [25]:
import numpy as np

In [26]:
symptoms_num = np.asarray(vector)

In [27]:
df.insert(11, 'symptoms_num', symptoms_num)

In [28]:
df.head()

,vn,sex,age_y,bps,bpd,pulse,bw,height,temperature,PDX,symptoms,symptoms_num,xray_items
0,610926001048,หญิง,52.0,170.0,110.0,64.0,86.0,154.0,36.6,K30,ปวด/แน่น/ปวดท้อง/แน่นใต้ลิ้นปี่,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Acute Abdomen
1,601013203924,หญิง,51.0,150.0,94.0,74.0,86.0,156.0,36.4,K30,เหนื่อย,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Chest PA Upright
2,610330191455,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,ปวด/ปวดเอว,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",LS-spine AP/lat/Acute Abdomen
3,610330191455,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,ปวด/ปวดเอว,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",LS-spine AP/lat/Acute Abdomen
4,610627072254,หญิง,61.0,124.0,74.0,64.0,76.0,155.0,36.0,M70,ปวด/ปวดเข่า/เจ็บ/-,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Knee Lt AP/lat standing/Knee Rt AP/lat standing


In [29]:
# vectorizer = TfidfVectorizer(tokenizer=slash_token)
vectorizer.fit(df['xray_items'])
vector_xray = vectorizer.transform(df['xray_items'])

/Users/phrare/patient_clustering/patient/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [30]:
xray_num = np.asarray(vector_xray)

In [31]:
df.insert(13, 'xray_num', xray_num)

In [32]:
df.head()

,vn,sex,age_y,bps,bpd,pulse,bw,height,temperature,PDX,symptoms,symptoms_num,xray_items,xray_num
0,610926001048,หญิง,52.0,170.0,110.0,64.0,86.0,154.0,36.6,K30,ปวด/แน่น/ปวดท้อง/แน่นใต้ลิ้นปี่,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Acute Abdomen,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
1,601013203924,หญิง,51.0,150.0,94.0,74.0,86.0,156.0,36.4,K30,เหนื่อย,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Chest PA Upright,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
2,610330191455,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,ปวด/ปวดเอว,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",LS-spine AP/lat/Acute Abdomen,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
3,610330191455,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,ปวด/ปวดเอว,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",LS-spine AP/lat/Acute Abdomen,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
4,610627072254,หญิง,61.0,124.0,74.0,64.0,76.0,155.0,36.0,M70,ปวด/ปวดเข่า/เจ็บ/-,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Knee Lt AP/lat standing/Knee Rt AP/lat standing,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."


In [40]:
df.drop(['vn', 'symptoms', 'xray_items'], axis = 1, inplace=True) 

,sex,age_y,bps,bpd,pulse,bw,height,temperature,PDX,symptoms_num,xray_num
0,หญิง,52.0,170.0,110.0,64.0,86.0,154.0,36.6,K30,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
1,หญิง,51.0,150.0,94.0,74.0,86.0,156.0,36.4,K30,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
2,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
3,หญิง,70.0,157.0,33.0,61.0,50.0,152.0,36.1,R1049,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
4,หญิง,61.0,124.0,74.0,64.0,76.0,155.0,36.0,M70,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
...,...,...,...,...,...,...,...,...,...,...,...
20252,หญิง,3.0,0.0,0.0,114.0,14.7,100.0,36.6,S400,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
20253,หญิง,3.0,0.0,0.0,114.0,14.7,100.0,36.6,S400,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
20254,ชาย,5.0,100.0,73.0,134.0,20.0,53.0,37.9,J40,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
20255,ชาย,14.0,123.0,79.0,92.0,45.0,160.0,37.1,S819,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....","(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."


In [41]:
# Split data into train and test
y = df.PDX
x = df.drop('PDX',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.head()

,vn,sex,age_y,bps,bpd,pulse,bw,height,temperature,symptoms,symptoms_num,xray_items,xray_num
17861,601107130426,ชาย,65.0,107.0,74.0,111.0,40.0,165.0,36.3,เหนื่อย/ไอ,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Chest PA Upright,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
1258,610131152602,ชาย,56.0,153.0,79.0,67.0,84.0,170.0,36.5,แน่นหน้าอก/สะอึก,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Abdomen supine/Abdomen upright,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
19263,610705100908,ชาย,31.0,98.0,66.0,96.0,50.0,160.0,36.0,ตัวเหลือง/ตาเหลือง/ถ่ายลำบาก,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Abdomen supine,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
3364,610205100956,หญิง,57.0,104.0,71.0,91.0,60.0,154.0,37.0,ปวดเมื่อย/ปวด,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Chest PA Upright,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."
12777,601124160433,ชาย,25.0,118.0,73.0,114.0,64.0,170.0,37.2,ทำร้ายร่างกาย,"(0, 226)\t0.7284197163691722\n (0, 222)\t0....",Skull AP/lat,"(0, 5)\t1.0\n (1, 24)\t1.0\n (2, 64)\t0.61..."


In [42]:
x_train.shape

(16205, 13)

In [48]:
type(x_train['symptoms_num'][1])

scipy.sparse.csr.csr_matrix

In [43]:
x_test.shape

(4052, 13)

In [44]:
from sklearn.svm import SVC

In [45]:
svclassifier = SVC()

In [46]:
svclassifier.fit(x_train, y_train)

ValueError: setting an array element with a sequence.